# Plotting Prep
This notebook takes in a variety of data inputs in order to complile a shapefile to be used in `Map Plotter.ipynb` for the purposes of creating maps and plots based on demographic data. **Please ensure you rename the output appropriately.**

### Package & Data Imports

In [ ]:
import geopandas as gpd
import pandas as pd
import shapely
import numpy as np
import matplotlib.pyplot as plt
import maup
from gerrychain import Graph

#Import native lands data
AIANHH_data = pd.read_csv("national_block_assignment_aianhh/national_block_assignment_aianhh.csv")
#Import results at block level data
national_2020_results_to_blocks = pd.read_csv("national_block_2020_pres_results/national_block_2020_pres_results.csv")
#Import block TIGER/Line file for target state
state_blocks = gpd.read_file("NewMexico/tl_2020_35_tabblock20/tl_2020_35_tabblock20.shp") 

/tmp/ipykernel_4575/3058630251.py:9: DtypeWarning: Columns (9,10,12,15,16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  AIANHH_data = pd.read_csv("national_block_assignment_aianhh/national_block_assignment_aianhh.csv")


In [ ]:
#Place P1 block level data for target state
state_racial_data = pd.read_csv("NewMexico/P1_NM_Data/DECENNIALPL2020.P1-Data.csv") 

/tmp/ipykernel_4575/3673418780.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  state_racial_data = pd.read_csv("NewMexico/P1_NM_Data/DECENNIALPL2020.P1-Data.csv") #Place P1 block level data for target state


Prepare for merging

In [ ]:
state_racial_data.drop(0, inplace=True) #Drop index
state_racial_data['GEO_ID'] = state_racial_data['GEO_ID'].str[9:26] #Reformat GEOID for merging
state_racial_data

,GEO_ID,NAME,P1_001N,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N,...,P1_063N,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N,Unnamed: 73
1,350010001071000,"Block 1000, Block Group 1, Census Tract 1.07, ...",343,307,289,0,0,15,0,3,...,0,0,0,0,0,0,0,0,0,NaN
2,350010001071001,"Block 1001, Block Group 1, Census Tract 1.07, ...",17,17,10,1,0,0,0,6,...,0,0,0,0,0,0,0,0,0,NaN
3,350010001071002,"Block 1002, Block Group 1, Census Tract 1.07, ...",46,39,30,0,4,4,0,1,...,0,0,0,0,0,0,0,0,0,NaN
4,350010001071003,"Block 1003, Block Group 1, Census Tract 1.07, ...",36,33,32,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
5,350010001071004,"Block 1004, Block Group 1, Census Tract 1.07, ...",35,23,19,0,0,1,0,3,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107211,350619803001004,"Block 1004, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
107212,350619803001005,"Block 1005, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
107213,350619803001006,"Block 1006, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
107214,350619803001007,"Block 1007, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


Convert to ints for merging prep

In [4]:
state_racial_data['GEOID20'] = state_racial_data['GEO_ID'].astype(int)
state_racial_data['P1_005N']=state_racial_data['P1_005N'].astype(int)
state_racial_data['P1_001N']=state_racial_data['P1_001N'].astype(int)
state_racial_data['P1_012N']=state_racial_data['P1_012N'].astype(int)
state_racial_data

,GEO_ID,NAME,P1_001N,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N,...,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N,Unnamed: 73,GEOID20
1,350010001071000,"Block 1000, Block Group 1, Census Tract 1.07, ...",343,307,289,0,0,15,0,3,...,0,0,0,0,0,0,0,0,NaN,350010001071000
2,350010001071001,"Block 1001, Block Group 1, Census Tract 1.07, ...",17,17,10,1,0,0,0,6,...,0,0,0,0,0,0,0,0,NaN,350010001071001
3,350010001071002,"Block 1002, Block Group 1, Census Tract 1.07, ...",46,39,30,0,4,4,0,1,...,0,0,0,0,0,0,0,0,NaN,350010001071002
4,350010001071003,"Block 1003, Block Group 1, Census Tract 1.07, ...",36,33,32,0,0,0,0,1,...,0,0,0,0,0,0,0,0,NaN,350010001071003
5,350010001071004,"Block 1004, Block Group 1, Census Tract 1.07, ...",35,23,19,0,0,1,0,3,...,0,0,0,0,0,0,0,0,NaN,350010001071004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107211,350619803001004,"Block 1004, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,350619803001004
107212,350619803001005,"Block 1005, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,350619803001005
107213,350619803001006,"Block 1006, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,350619803001006
107214,350619803001007,"Block 1007, Block Group 1, Census Tract 9803, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,350619803001007


In [ ]:
state_blocks['GEOID20']=state_blocks['GEOID20'].astype(int)

### Begin Merging

In [ ]:

state_blocks = state_blocks.merge(AIANHH_data, on = ['GEOID20'], how = 'left')
state_blocks = state_blocks.merge(national_2020_results_to_blocks, on = ['GEOID20'], how = 'left')
state_blocks = state_blocks.merge(state_racial_data, on = ['GEOID20'], how = 'left')


#### Create ercentages

In [6]:

def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return 0
state_blocks['native_american_perc'] = divide(state_blocks['P1_005N'],state_blocks['P1_001N'])
state_blocks['mixed_native_american_perc'] = divide(state_blocks['P1_012N'],state_blocks['P1_001N'])
state_blocks['perc_trump'] = state_blocks['G20PRERTRU']/(state_blocks['G20PRERTRU'] + state_blocks['G20PREDBID'])
state_blocks

,STATEFP20_x,COUNTYFP20_x,TRACTCE20_x,BLOCKCE20_x,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N,Unnamed: 73,native_american_perc,mixed_native_american_perc,perc_trump
0,35,035,000305,2014,350350003052014,Block 2014,G5040,U,00820,U,...,0,0,0,0,0,0,NaN,0.000000,0.028571,0.637025
1,35,035,000904,4521,350350009044521,Block 4521,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,35,035,000602,1006,350350006021006,Block 1006,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,35,035,000904,4399,350350009044399,Block 4399,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
4,35,035,000701,2007,350350007012007,Block 2007,G5040,R,None,None,...,0,0,0,0,0,0,NaN,0.052632,0.157895,0.616999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107210,35,053,978100,1409,350539781001409,Block 1409,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
107211,35,045,000706,2143,350450007062143,Block 2143,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
107212,35,015,000900,1666,350150009001666,Block 1666,G5040,R,None,None,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
107213,35,053,978200,2066,350539782002066,Block 2066,G5040,R,None,None,...,0,0,0,0,0,0,NaN,0.000000,0.272727,0.588452


In [ ]:
state_blocks['NALAND'] = state_blocks['AIANHH_NAME'] #Renaming column to something shorter for export. See note at bottom.
state_blocks.drop('AIANHH_NAME',inplace=True,axis=1)


In [9]:
state_blocks['NALAND']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
107210    NaN
107211    NaN
107212    NaN
107213    NaN
107214    NaN
Name: NALAND, Length: 107215, dtype: object

### Export
**NOTE:** Exporting cuts off column names at 10 characters, so it might be wise to rename column names to something shorter before exporting. 

In [10]:
state_blocks.to_file("Plottables/NM")

/tmp/ipykernel_4575/2715408969.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  state_blocks.to_file("Plottables/NM")
